# Linear Regression 

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from module import NaN_checker, standardizer, NaN_remover, extract_features_from_mol, unbiased_RT, features_reduction_using_correlation, correlation_matrix, enrich, PrincipalComponentAnalysis
from rdkit import Chem
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



In [2]:
train = pd.read_csv('train_scaled.csv')
enriched_train = enrich(pd.read_csv('train.csv'))
train_cddd_merged = pd.read_csv('train_cddd_merged.csv')
cddd = pd.read_csv('cddd_scaled.csv')

We create a function to perform linear regression on our datasets. 

In [5]:
def linearregression(xtrain, ytrain, xtest, ytest = pd.DataFrame()): 
    """
    performing linear regression on a Dataframe. 
    
    Parameters : 
    - xtrain (pd.dataframe) : Dataframe containing features values to train the model on.
    - ytrain (pd.dataframe) : Dataframe containing outcome variable values to train de model on.
    - xtest (pd.dataframe) : Dataframe containing features values to test the model on. 
    - ytest (pd.dataframe) : Datadrame containing outcome variable values to tesz the model on.
    
    Returns : 
    - train_mse : mean squared error between the outcome values estimated by the model and the real outcome values in the training set (ytrain).
    - test_mse : mean squared error between the outcome values estimated by the model and the real outcome values in the test set (ytest).
    - coefs : Dataframe containing the coefficients found by the model for each feature. 
    - RT : numpy.ndarray of the values predicted by the model.
    """
    m = LinearRegression()
    m.fit(xtrain,ytrain)
    coefs = pd.DataFrame({
	    'predictor': xtrain.columns.tolist(),
	    'value': m.coef_[0]
	    })
    RT = m.predict(xtest)
    train_mse = mean_squared_error(m.predict(xtrain), ytrain) ** 0.5
    if ytest.empty: 
        test_mse = None
    else : 
        test_mse = mean_squared_error(m.predict(xtest), ytest) ** 0.5
    return train_mse, test_mse, coefs, RT

Here we apply the function to our four different datasets. 

In [4]:
xtrain = train.iloc[:1000].drop(['Compound', 'SMILES', 'Lab', 'mol','RT'], axis=1)
ytrain = train['RT'].iloc[:1000]
xtest = train.iloc[1001:].drop(['Compound', 'SMILES', 'Lab', 'mol','RT'], axis=1)
ytest = train['RT'].iloc[1001:]

train_mse, test_mse, coefs, RT = linearregression(xtrain, ytrain, xtest, ytest)
print(train_mse)
print(test_mse)
print(RT)

1.6944561240305511
32648251296803.21
[-1.18398326e+14  5.55139160e+00  1.06051025e+01 ...  1.29775391e+01
 -1.89780498e+13  7.07922363e+00]


In [5]:
xtrain = enriched_train.iloc[:1000].drop(['Compound', 'SMILES', 'Lab','RT', 'mean_RT', 'Bias', 'Corrected_RT'], axis=1)
ytrain = enriched_train['RT'].iloc[:1000]
xtest = enriched_train.iloc[1001:].drop(['Compound', 'SMILES', 'Lab','RT', 'mean_RT', 'Bias','Corrected_RT'], axis=1)
ytest = enriched_train['RT'].iloc[1001:]

train_mse, test_mse, coefs, RT = linearregression(xtrain, ytrain, xtest, ytest)
print(train_mse)
print(test_mse)
print(RT)

0.6801276862081973
1570336196062.9834
[ 1.34727478e+01  4.90044403e+00 -9.54943992e+09 ...  1.91985497e+10
  2.74422892e+12  5.53390503e+00]


In [6]:
xtrain = train_cddd_merged.iloc[:1000].drop(['Compound', 'SMILES', 'Lab', 'mol','RT'], axis=1)
ytrain = train_cddd_merged['RT'].iloc[:1000]
xtest = train_cddd_merged.iloc[1001:].drop(['Compound', 'SMILES', 'Lab', 'mol','RT'], axis=1)
ytest = train_cddd_merged['RT'].iloc[1001:]

train_mse, test_mse, coefs, RT = linearregression(xtrain, ytrain, xtest, ytest)
print(train_mse)
print(test_mse)
print(RT)

1.5404516607900411
27543149063608.527
[ 1.03959961e+01  6.19396973e+00  3.62496319e+10 ...  2.50203437e+12
 -4.54277372e+13  3.25769043e+00]


In [7]:
withoutcddd = pd.read_csv('train_scaled.csv')
withoutcddd = extract_features_from_mol(withoutcddd)
xtrain = withoutcddd.iloc[:1000].drop(['Compound', 'SMILES', 'Lab','RT'], axis=1)
ytrain = withoutcddd['RT'].iloc[:1000]
xtest = withoutcddd.iloc[1001:].drop(['Compound', 'SMILES', 'Lab','RT'], axis=1)
ytest = withoutcddd['RT'].iloc[1001:]

train_mse, test_mse, coefs, RT = linearregression(xtrain, ytrain, xtest, ytest)
print(train_mse)
print(test_mse)
print(RT)


1.688987202033922
115513721801.30844
[-1.00235246e+11  5.65585327e+00  1.06555252e+01 ...  1.27994690e+01
  3.80443493e+11  7.08302307e+00]


We see that we have a huge test mean squared error. It probably due to the huge number of features that we have.

## Finding the best combination of parameters
We want to find the best combination of parameters to have the smaller test mean squared error. 
First, we define a function to evaluate the mean squared error for a list of given predictors. 

In [8]:
def run_and_evaluate(train, test, predictor_names, target = 'RT'):
    """
    Evaluate the test mean squared error of a train and a test Dataframes for a given list of predictors
    
    Parameters : 
    - train (pd.dataframe) : Dataframe containing the training data for the linear model.
    - test (pd.dataframe) : Dataframe containing the test data which will serve to estimate the mean squared error of the model. 
    - perdictor_names (pd.dataframe) : Dataframe containing a list of the predictors used by the linear model.
    
    Returns : 
    - np.array : containing the mean squared error for the prediction with the predictors given.
    """
    model = LinearRegression().fit(train[predictor_names].values,
                                   train[target].values)
    y_pred = model.predict(test[predictor_names].values)
    return np.sqrt(mean_squared_error(test[target].values, y_pred))

Then, we define a function that returns the names of the best predictors to have the smalest test mean squared error. 

In [9]:
def best_predictors(train, test, target = ['RT']) : 
    """
    Find the best combination of two predictors to predict the target value in a Dataframe.
    
    Parameters : 
    - train (pd.dataframe) : Dataframe containing the training data for the linear model.
    - test (pd.dataframe) : Dataframe containing the test data which will serve to estimate the mean squared error of the model. 
    - target : the target value we are trying to predict with the given predictors. 
    
    Returns : 
    - pd.dataframe : containing the mean squared error of each pairs of predictors.
    """
    predictors = [col for col in train.columns if col not in target]
    predictor_pairs = [[p1, p2] for p1 in predictors for p2 in predictors if p1 > p2]
    results = [(run_and_evaluate(train, test, p), ", ".join(p)) for p in predictor_pairs]
    results_df = pd.DataFrame(results, columns=["test_loss", "predictors"]).sort_values(by="test_loss").reset_index(drop=True)
    return results_df
    
    

We now apply our function to the enriched Dataframe. (result : ECFP_888 and ECFP_334 not really reliable as overfitting of the sample maybe)

In [10]:
training = enriched_train.iloc[:1000].drop(['Compound', 'SMILES', 'Lab', 'mean_RT', 'Bias', 'Corrected_RT'], axis=1)
testing = enriched_train.iloc[1001:].drop(['Compound', 'SMILES', 'Lab', 'mean_RT', 'Bias','Corrected_RT'], axis=1)

bp = best_predictors(train, test)

print(bp)

KeyboardInterrupt: 

We naively try to do a linear regression with the features appearing high in the classement to see if it improves the mean squared error. 

In [11]:
xtrain = enriched_train[['cddd_432', 'cddd_458', 'CarbonAtoms', 'TotalAtoms', 'MolLogP', 'lab_mean_bias']].iloc[:1000]
ytrain = enriched_train['RT'].iloc[:1000]
xtest = enriched_train[['cddd_432', 'cddd_458', 'CarbonAtoms', 'TotalAtoms', 'MolLogP', 'lab_mean_bias']].iloc[1001:]
ytest = enriched_train['RT'].iloc[1001:]

train_mse, test_mse, coefs, RT = linearregression(xtrain, ytrain, xtest, ytest)
print(train_mse)
print(test_mse)
print(RT)

1.7858833887296208
1.7944313743025826
[12.98461199  5.95069099  3.32716166 ...  9.98875197  6.82005831
  5.3973023 ]


We see that the mean squared error is clearly better but also because we have less parameters.

## Reducing features using Principal Component Analysis 

In [4]:
PCA_train = PrincipalComponentAnalysis(enriched_train, 31)
print(PCA_train.shape)
print(PCA_train.columns)

(3500, 35)
Index(['Compound', 'SMILES', 'Lab', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6',
       'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15',
       'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24',
       'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'RT'],
      dtype='object')


We try to do a linear regression with the obtained dataset after Principale Component Analysis (keeping percentage = 31%)

In [17]:
xtrain = PCA_train.drop(['Compound', 'SMILES', 'Lab','RT'], axis = 1).iloc[:1000]
ytrain = PCA_train['RT'].iloc[:1000]
xtest = PCA_train.drop(['Compound', 'SMILES', 'Lab', 'RT'], axis = 1).iloc[1001:]
ytest = PCA_train['RT'].iloc[1001:]

train_mse, test_mse, coefs, RT = linearregression(xtrain, ytrain, xtest, ytest)
print(train_mse)
print(test_mse)
print(RT)

1.8934905906695358
1.9897580765967073
[12.98986074  5.36538892  5.22047815 ... 10.34828786  6.49447991
  3.98856037]


We have an improvement but not as good as when we used naively the predictors in the best pairs of predictors (maybe because we have more predictors (we could try with a lower keeping percentage)).

We are also going to try the last non zero features when performing Lasso regularisation. 

In [7]:
xtrain = enriched_train[['ECFP_41','ECFP_46','ECFP_152','ECFP_262','ECFP_334','ECFP_491','ECFP_550','ECFP_592','ECFP_882','ECFP_888',
                        'ECFP_990', 'MolecularWeight','TotalAtoms','CarbonAtoms','MolLogP','cddd_9','cddd_14','cddd_27','cddd_52','cddd_74',
                        'cddd_159','cddd_237','cddd_251','cddd_370','cddd_374','cddd_380','cddd_410','cddd_450', 'cddd_508']].iloc[:1000]
ytrain = enriched_train['RT'].iloc[:1000]
xtest = enriched_train[['ECFP_41','ECFP_46','ECFP_152','ECFP_262','ECFP_334','ECFP_491','ECFP_550','ECFP_592','ECFP_882','ECFP_888',
                        'ECFP_990', 'MolecularWeight','TotalAtoms','CarbonAtoms','MolLogP','cddd_9','cddd_14','cddd_27','cddd_52','cddd_74',
                        'cddd_159','cddd_237','cddd_251','cddd_370','cddd_374','cddd_380','cddd_410','cddd_450', 'cddd_508']].iloc[1001:]
ytest = enriched_train['RT'].iloc[1001:]

train_mse, test_mse, coefs, RT = linearregression(xtrain, ytrain, xtest, ytest)
print(train_mse)
print(test_mse)
print(RT)


2.5679599423272075
2.616480099093414
[ 8.35390613  6.87708716  7.85501341 ... 10.83102655  5.32727484
  4.68512673]


The PCA gives better results so we are going to stick to that for now. 

We are going to try the linear regression combined with a lasso regularisation. We found in the regularisation notebook that, when using the enriched train dataset, the best alpha was 0.1.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(enriched_train.drop(['Compound', 'SMILES', 'Lab','RT'], axis = 1), enriched_train['RT'], test_size=0.2, random_state=42)

alpha = 0.1  
lasso_model = Lasso(alpha=alpha)

lasso_model.fit(X_train, y_train)

predictions = lasso_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

non_zero_coefficients = np.sum(lasso_model.coef_ != 0)
print(f'Nombre de coefficients non nuls : {non_zero_coefficients}')

Mean Squared Error: 6.990276765924979
Nombre de coefficients non nuls : 89


Now we try to combine the lasso regularisation with the PCA features. 

In [5]:
X_train, X_test, y_train, y_test = train_test_split(PCA_train.drop(['Compound', 'SMILES', 'Lab','RT'], axis = 1), PCA_train['RT'], test_size=0.2, random_state=42)

alpha = 0.1  
lasso_model = Lasso(alpha=alpha)

lasso_model.fit(X_train, y_train)

predictions = lasso_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

non_zero_coefficients = np.sum(lasso_model.coef_ != 0)
print(f'Nombre de coefficients non nuls : {non_zero_coefficients}')

Mean Squared Error: 4.5301135523902065
Nombre de coefficients non nuls : 30


## Basic linear regression kaggle submission
(We have seen that, strangely, if we added by mistake a second, not scaled cddd to the enriched train data set, the mean squared error was even lower.) To be able to submit our estimations to kaggle, we have to prepare our test set the same way our training set (enriched train) was prepared. Using the enrich function. The fonction will extract features from the mol column, add the cddd features and replace the NaN in the lines where no cddd was given (no corresponding SMILES) by the mean of their column. 

In [5]:
test = pd.read_csv('test.csv')

In [10]:
enriched_test = enrich(test)
print(enriched_test)


                     Compound  \
0               5Cl-AB-PINACA   
1                  SDB-006-5F   
2                  Cephaeline   
3     JWH-081 (4-OH-Naphthyl)   
4                      5F-AEB   
...                       ...   
1370      4-methylamphetamine   
1371                  Cocaine   
1372               Clonazolam   
1373       1-Hydroxymidazolam   
1374            Phenylephrine   

                                                 SMILES  \
0            CC(C)C(NC(=O)c1nn(CCCCCCl)c2ccccc12)C(N)=O   
1                  O=C(NCc1ccccc1)c1cn(CCCCCF)c2ccccc12   
2     CCC1CN2CCc3cc(OC)c(OC)cc3C2CC1CC1NCCc2cc(O)c(O...   
3            CCCCCn1cc(C(=O)c2ccc(O)c3ccccc23)c2ccccc21   
4           CCOC(=O)C(NC(=O)c1nn(CCCCCF)c2ccccc12)C(C)C   
...                                                 ...   
1370                                  Cc1ccc(CC(C)N)cc1   
1371               COC(=O)C1C(OC(=O)c2ccccc2)CC2CCC1N2C   
1372  Cc1nnc2n1-c1ccc([N+](=O)[O-])cc1C(c1ccccc1Cl)=NC2   
1373         

We also have to add the laboratory bias to our test set.

Now that our test set is ready, we are going to have to train our model on the full training set. 